# Download Low Rate Data Logger Files From AWS S3 - Test

Download all files from a single logger and concatenate to a single file for analysis

In [60]:
import boto3
import os
import glob
import shutil

In [69]:
bucket = 'dev-data-logger-lake.geonet.org.nz'

dltmp = '/home/sherburn/scratch/devlogger/tmp'
dlsav = '/home/sherburn/scratch/devlogger'

loggers = ['infernocratertest', 'lowertemaari']

In [62]:
s3 = boto3.client('s3')

In [70]:
list=s3.list_objects_v2(Bucket=bucket)['Contents']

for logger in loggers:
    os.makedirs(dltmp, exist_ok=True) #make tmp directory for downloaded files
    for key in list:
        if ((logger in key['Key']) and ('Table1' in key['Key'])):
            #print (key['Key'])
            s3.download_file(bucket, key['Key'], os.path.join(dltmp, os.path.basename(key['Key'])))

    #concat all files
    concatfile = os.path.join(dlsav, logger+'_Table1.csv')
    files = glob.glob(os.path.join(dltmp, '*.csv'))
    with open(concatfile, 'w') as outfile:
        for file in files:
            with open(file, 'r') as readfile:
                shutil.copyfileobj(readfile, outfile)

    shutil.rmtree(dltmp)#remove tmp directory for downloaded files